In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import os

Mounted at /content/drive


In [ ]:
# Putting all the features that PubMed returns into a pandas dataframe ...

from pathlib import Path
# csv_file = Path("/content/drive/MyDrive/initial_df.csv")

# if csv_file.is_file():
#   df = pd.read_csv(str(csv_file.resolve()))
#   df = df.astype(str)
#   print( "Initial Dataframe retrieved from \"initial_df.csv\"!" )
# else:
  # Show number of duplicated PMID values:

import glob

csv_df = pd.DataFrame(columns=[])

# for file in glob.glob("/content/drive/MyDrive/changed_CD012768*.csv"): # tested the whole thing only on "changed_CD012768.csv" and "changed_CD012768 (copy).csv" ...
for file in glob.glob("/content/drive/MyDrive/DTA/*/*.csv"):
  filename = os.path.basename(file)
  temp_df = pd.read_csv(file, index_col=0)
  temp_df[ "Filename" ]=filename
  if ( "'int'" in str(type(csv_df)) ):
    csv_df = temp_df.copy()
  else:
    csv_df = pd.concat([csv_df, temp_df], axis=0, ignore_index=True)
  print( "All " + str(temp_df.shape[0]) + " rows of \"" + file + "\" were added to \"csv_df\" .. " )
  print( "\"csv_df\" has " + str(csv_df.shape[0]) + " rows ..." )
  # csv_df = csv_df.drop_duplicates()
  # print( "\"csv_df\" has " + str(csv_df.shape[0]) + " rows after duplicate removal ..." )
  print( "===========" )

print( "Total count of of \"PMID\" values:\t" + str(csv_df.shape[0]) )
print( "Duplicate count of \"PMID\" column:\t" + str(csv_df.duplicated(subset="PMID").sum()) )

pmids = list(csv_df["PMID"])
pmids = list(dict.fromkeys(pmids)) # remove duplicate PMID values
print( "Number of distinct \"PMID\" values:\t" + str(len(pmids)) )


# Shows duplicate count for each column

# for column in csv_df.columns.values:
#   print( "Duplicate count of \"" + column + "\" column: ", end='' )
#   print( csv_df.duplicated(subset=column).sum() )

All 119 rows of "/content/drive/MyDrive/DTA/CD012768/finished_new_prompt_CD012768.csv" were added to "csv_df" .. 
"csv_df" has 119 rows ...
All 119 rows of "/content/drive/MyDrive/DTA/CD012768/ada_embed_CD012768.csv" were added to "csv_df" .. 
"csv_df" has 238 rows ...
All 119 rows of "/content/drive/MyDrive/DTA/CD012768/finished_new_prompt_CD012768_hard.csv" were added to "csv_df" .. 
"csv_df" has 357 rows ...
All 119 rows of "/content/drive/MyDrive/DTA/CD012768/finished_new_prompt_CD012768_soft.csv" were added to "csv_df" .. 
"csv_df" has 476 rows ...
All 1799 rows of "/content/drive/MyDrive/DTA/CD008874/ada_embed_CD008874.csv" were added to "csv_df" .. 
"csv_df" has 2275 rows ...
All 1799 rows of "/content/drive/MyDrive/DTA/CD008874/finished_new_prompt_CD008874.csv" were added to "csv_df" .. 
"csv_df" has 4074 rows ...
All 1799 rows of "/content/drive/MyDrive/DTA/CD008874/finished_new_prompt_CD008874_soft.csv" were added to "csv_df" .. 
"csv_df" has 5873 rows ...
All 3120 rows of "/

In [ ]:
 # Imports and so on ...

  !pip install metapub
  import numpy as np
  pd.set_option('display.max_columns', None) # so it prints all the dataframe columns ...
  from metapub import PubMedFetcher
  import json
  fetch = PubMedFetcher()

  def hasmethod(obj, name): # the functions returned should be stored separately ... may be used to return something that is not in the other properties later on ...
      return hasattr(obj, name) and ( "method" in str(type(getattr(obj, name))) )

  all_article_data=[]
  column_names=[]
  function_names=[]
  for i in range( 0, len( pmids ) ):
    try:
      article = fetch.article_by_pmid(pmids[i])
      article_data = {}
      for attr in dir( article ):
        if ( i == 0 ):
          if ( not hasmethod( article, str( attr ) ) ):
              column_names.append( attr )
          else:
              function_names.append( attr )
        if ( not hasmethod( article, str( attr ) ) ):
          article_data[ attr ] = getattr(article, attr)
      all_article_data.append( article_data )
    except:
      pass

  df = pd.DataFrame( all_article_data, columns=column_names )
  df.to_csv("/content/drive/MyDrive/initial_df.csv", encoding='utf-8', index=False)
  print( "Initial Dataframe saved to \"initial_df.csv\"!" )

In [ ]:
# Dropping all columns that only have NaN or fully identical values ...

count_row = df.shape[0]  # Gives number of rows
freq_df=df.describe().loc[['freq']]
count_df=df.describe().loc[['count']]
for column in freq_df:
    if ( freq_df[column].iloc[0] == count_df[column].iloc[0] ):
      df = df.drop(column, axis=1)
    if ( pd.isna(freq_df[column].iloc[0]) ):
      df = df.drop(column, axis=1)

In [ ]:
# This whole thing was a xml to json conversion that barely helped at all ... better to just parse the xml ...

# import xml.etree.ElementTree as ET
# all_json_xml_tags=[]

# def extract_row_xml_tags(row):
#   xml_tree = ET.ElementTree(ET.fromstring(row["xml"]))
#   all_xml_tags = []
#   for elem in xml_tree.iter():
#       all_xml_tags.append(elem.tag)
#   all_xml_tags = list(set(all_xml_tags))
#   # print(all_xml_tags)

#   xml_tag_json={}
#   for elem in all_xml_tags:
#     final_arr=[]
#     arr = xml_tree.findall(".//"+elem)
#     for i in arr:
#       # print( ET.tostring( i, encoding='unicode' ) )
#       final_arr.append( ET.tostring(i, encoding='unicode') )
#     xml_tag_json[ elem ] = final_arr

#   json_string = json.dumps( xml_tag_json )
#   return json_string

# df["json_xml"] = df.apply(extract_row_xml_tags, axis=1)

# import json
# print( json.dumps(df.iloc[0]["json_xml"], indent=4) )

In [ ]:
# This reveals that the content feature is contained in the xml feature (they are both xml representations of the same thing)

# from lxml import etree
# content=df.iloc[0]["content"]
# etree.tostring(content, pretty_print=True)

In [ ]:
# Removing more useless columns and adding some new ones to better describe the data ...

remove_columns=[ "__dict__", "pii", "author1_lastfm", "author_list", "authors_str", "author1_last_fm", "content", "citation_html", "pages", "first_page", "last_page" ]
# pcim / doi are redundant as they are found in the urls too ... yet they were not removed for now ...
# pii contains the issn and other things ... no way to retrieve article though "Publisher Item Identifier", unless you go to the actual publisher's website ...

if "author1_last_fm" in df.columns:
  df["author_first"]=df["author1_last_fm"]

# if "pmc" in df.columns: # so as to parse the webpage like with the pubmed site ...
#   df['pmc_url']="https://pmc.ncbi.nlm.nih.gov/articles/PMC"+str(df['pmc'])+"/"




# Uncomment this to keep the doi in an url ... yet, each doi url points to a different 2nd domain, which is NOT the same for all articles, meaning that it is not easy to get data from it (website structures differ) ...

# if "doi" in df.columns: # so as to parse the webpage like with the pubmed site ...
#   df["doi_url"]="https://doi.org/"+df["doi"]


for i in remove_columns:
  if i in df.columns:
    df = df.drop(i, axis=1)
df.describe()


# Choose one of the following methods for formatting xml in a visible way ... Quite sure the first one has no bugs ...

# import xml.dom.minidom
# def restyle_xml(row):
#   return xml.dom.minidom.parseString(row["xml"]).toprettyxml()
# df['xml'] = df.apply(restyle_xml, axis=1)
# print( df.iloc[0]["xml"] )

# from lxml import etree
# def restyle_xml(row):
#   x = etree.fromstring(row["xml"])
#   return etree.tostring(x, pretty_print=True)
# df['xml'] = df.apply(restyle_xml, axis=1)

,abstract,authors,chemicals,citation,doi,history,issn,issue,journal,keywords,...,pmc,pmid,publication_types,title,url,volume,volume_issue,xml,year,author_first
count,26806,26806,26806,26806,26806,26806,26806,26806,26806,26806,...,26806,26806,26806,26806,26806,26806,26806,26806,26806,26806
unique,26797,26376,5887,26806,21320,19048,4193,470,3323,2491,...,4388,26806,528,26717,26806,639,4324,26806,49,20670
top,CURRENT DIAGNOSTIC METHODS: Helicobacter pylor...,[],{},"Peter JG, et al. The diagnostic accuracy of ur...",nan,"{'pubmed': datetime.datetime(1996, 1, 1, 0, 0)...",1573-3432,1,Spine (Phila Pa 1976),[],...,nan,22815718,{'D016428': 'Journal Article'},Diagnosis of Helicobacter pylori infection.,https://ncbi.nlm.nih.gov/pubmed/22815718,18,nan,"b'<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArti...",2013,Gisbert JP
freq,2,10,17051,1,5487,100,489,3555,583,24312,...,22419,1,7777,10,1,472,208,1,1891,72


In [ ]:
df.to_csv("/content/drive/MyDrive/checkpoint_1_df.csv", encoding='utf-8', index=False)

In [ ]:
from ast import literal_eval
df['xml'] = df['xml'].apply(literal_eval) # converts string to list ... only way ...

In [ ]:
# Getting references with ids from the "xml" property that PubMed returned ...

import xml.etree.ElementTree as ET

def get_references(row):
  xml_tree = ET.ElementTree(ET.fromstring(row["xml"]))
  have_citation=0
  for elem in xml_tree.iter():
    if ( "'ReferenceList'" in str( elem ) ):
      have_citation=1
      break

  # <Reference>
  # 	<Citation>REFERENCE ARTICLE TITLE</Citation>
  # 	<ArticleIdList>
  # 		<ArticleId IdType="pmc">PMC ID</ArticleId>
  # 		<ArticleId IdType="pubmed">PUBMED ID</ArticleId>
  # 	</ArticleIdList>
  # </Reference>

  all_citations=[]
  if ( have_citation == 1 ):
    ref_list=xml_tree.find(".//ReferenceList")
    refs=ref_list.findall(".//Reference")
    for ref in refs:
      citation_json={}
      for citation in ref.findall( ".//Citation" ):
        citation_json[ "citation" ] = citation.text
      article_id_list=ref.find( ".//ArticleIdList" )

      article_id_json={}
      if ( article_id_list != None ):
        for article_id in article_id_list.findall( ".//ArticleId" ):
          article_id_json[ article_id.attrib['IdType'] ] = article_id.text

      citation_json[ "article_ids" ] = article_id_json
      all_citations.append( citation_json )

  return json.dumps( all_citations )

df['references'] = df.apply(get_references, axis=1)

In [ ]:
citation_df = df[["pmid", "references"]]
citation_df = citation_df.astype(str)
citation_df = citation_df[citation_df["references"] != "[]" ]
citation_df.to_csv("/content/drive/MyDrive/citations.csv", encoding='utf-8', index=False)

In [ ]:
from ast import literal_eval
df['keywords'] = df['keywords'].apply(literal_eval) # converts string to list ... only way ...

In [ ]:
# Here, since most pmids do NOT have any keywords, some are generated for each of them ...

# This is done by:
#   0. removing all the common words from the title (done using lists of common words from wiki)
#   1. going though all the keywords that were found (from the articles that had them), to make a list of non-generated keywords/phrases
#   2. adding non-generated keywords/phrases to an article without keywords if they appear in its title
#   3. adding the final non-common title words to the keywords (the ones that remained = were not common and were not in non-generated keywords/phrases)
# (for everything to function properly everything has to be lowercase)

# All the common words are saved to a csv file, for easy access ...

from pathlib import Path
csv_file = Path("/content/drive/MyDrive/stop_words.csv")

if csv_file.is_file():
  print( "Stop words retrieved from \"stop_words.csv\"!" )
  stop_words = list(pd.read_csv(str(csv_file.resolve()))["word"])
else:
  print( "Stop words were re-generated and saved to \"stop_words.csv\"!" )
  import nltk
  nltk.download('stopwords')
  stop_words = list(stopwords.words("english"))

  print( "length before wiki: " + str( len(stop_words) ) )
  import urllib.request
  from bs4 import BeautifulSoup

  fp = urllib.request.urlopen("https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/English/Wikipedia_(2016)")
  mybytes = fp.read()
  html_page = mybytes.decode("utf8")
  fp.close()
  soup = BeautifulSoup(html_page)
  all_p = soup.find_all("p")
  word_p = []
  for p in all_p:
    if ( len( p.find_all() ) > 100 ):
      word_p.append( p )

  for p in word_p:
    word_a = list( p.find_all() );
    # print( word_a )
    for a in word_a:
        stop_words.append( a.text.lower() )

  print( "length after wiki 1: " + str( len(stop_words) ) )
  stop_words = list(dict.fromkeys(stop_words)) # removing duplicate words
  print( "length after wiki 1 (duplicates removed): " + str( len(stop_words) ) )
  fp = urllib.request.urlopen("https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/PG/2005/10/1-10000")
  mybytes = fp.read()
  html_page = mybytes.decode("utf8")
  fp.close()
  soup = BeautifulSoup(html_page)
  all_p = soup.find_all("p")
  word_p = []
  for p in all_p:
    if ( len( p.find_all() ) > 100 ):
      word_p.append( p )

  for p in word_p:
    word_a = list( p.find_all("a"));
    # print( word_a )
    for a in word_a:
      stop_words.append( a["title"].lower() )

  print( "length after wiki 2: " + str( len(stop_words) ) )
  stop_words = list(dict.fromkeys(stop_words)) # removing duplicate words
  print( "length after wiki 2 (duplicates removed): " + str( len(stop_words) ) )

  def add_more_stop_words (page):
    global stop_words
    fp = urllib.request.urlopen("https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/TV/2006/"+page)
    mybytes = fp.read()
    html_page = mybytes.decode("utf8")
    fp.close()
    soup = BeautifulSoup(html_page)
    all_td = soup.find_all("td")
    for td in all_td:
      if ( td.find("a") != None ):
        stop_words.append( td.find("a")["title"].lower() )
    stop_words = list(dict.fromkeys(stop_words)) # removing duplicate words


  for page_iter in range( 0, 10000, 1000 ):
    # print( str( page_iter + 1 ) + "-" + str( page_iter + 1000 ) )
    add_more_stop_words( str( page_iter + 1 ) + "-" + str( page_iter + 1000 ) )
  for page_iter in range( 10000, 20000, 2000 ):
    # print( str( page_iter + 1 ) + "-" + str( page_iter + 2000 ) )
    add_more_stop_words( str( page_iter + 1 ) + "-" + str( page_iter + 2000 ) )
  add_more_stop_words( "40001-41284" )


  print( "length after wiki 3: " + str( len(stop_words) ) )
  stop_words = list(dict.fromkeys(stop_words)) # removing duplicate words
  print( "length after wiki 3 (duplicates removed): " + str( len(stop_words) ) )

  stop_words_df = pd.DataFrame( stop_words, columns=["word"] )
  stop_words_df.to_csv("/content/drive/MyDrive/stop_words.csv", encoding='utf-8', index=False)



def get_keywords(sentence):
  words = sentence.lower().split()
  filtered_words = [word for word in words if word not in stop_words]
  uncommon_words = ' '.join(filtered_words)

  valid_characters = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ\- /_0123456789')
  # as such: "!"#$%&'()*+,.:;<=>?@[\\]^`{|}~" are now allowed of keyword names

  final_string = ''.join(c for c in uncommon_words if c in valid_characters)
  return final_string



count_empty=0
count_non_empty=0
non_generated_keywords=[]
def determine_keywords(row):
  global non_generated_keywords
  if ( row["keywords"] != [] ):
    row["keywords"]=[x.lower() for x in row["keywords"]]
    global count_non_empty
    count_non_empty = count_non_empty + 1
    non_generated_keywords.extend(row["keywords"])
    # print( row["title"] )
    # print( str( row["keywords"] ) + "\n\n" )
  non_generated_keywords = sorted(non_generated_keywords, key=len, reverse=True) # sorting by length
  non_generated_keywords = list(dict.fromkeys(non_generated_keywords)) # removing duplicate keywords



def generate_keywords(row):
  if ( row["keywords"] == [] ):
    global count_empty
    global non_generated_keywords
    count_empty = count_empty + 1
    # print( row["title"] )
    keywords=[]
    generated_keywords = get_keywords( row["title"] )
    for i in range(0, len( non_generated_keywords ) ):
      if ( non_generated_keywords[i] in generated_keywords ):
        generated_keywords = generated_keywords.replace( non_generated_keywords[i], "" )
        keywords.append( non_generated_keywords[i] )
    keywords.extend( generated_keywords.split( ' ' ) )
    keywords=[x.lower() for x in keywords]
    valid_characters = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ\\- /_0123456789')
    # as such: "!"#$%&'()*+,.:;<=>?@[\\]^`{|}~" are now allowed of keyword names
    for keyword in keywords:
      keyword = ''.join(c for c in keyword if c in valid_characters)

    for keyword in keywords:
      for i in range(0, len(keyword)):
        if keyword[i] in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789":
          break
        if keyword[i] in "\\- /_":
          if ( keyword in keywords ):
            keywords.remove(keyword)
          keywords.append(keyword[1:])
      for i in range(len(keyword)-1, -1, -1):
        if keyword[i] in "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789":
          break
        if keyword[i] in "\\- /_":
          if ( keyword in keywords ):
            keywords.remove(keyword)
          keywords.append(keyword[1:])

    while '' in keywords:
        keywords.remove('')

    return str( keywords )
  else:
    return str( row["keywords"] )

non_generated_keywords_file = Path("/content/drive/MyDrive/non_generated_keywords.csv")
if non_generated_keywords_file.is_file():
  non_generated_keywords = list(pd.read_csv(str(non_generated_keywords_file.resolve()))["keyword"])
  print( "Non-generated keywords retrieved from \"non_generated_keywords.csv\"!" )
else:
  df.apply(determine_keywords, axis=1)
  non_generated_keywords_df = pd.DataFrame( non_generated_keywords, columns=["keyword"] )
  non_generated_keywords_df.to_csv("/content/drive/MyDrive/non_generated_keywords.csv", encoding='utf-8', index=False)
  print( "Non-generated keywords were re-generated and saved to \"non_generated_keywords.csv\"!" )

df["keywords"]=df.apply(generate_keywords, axis=1)
print( "Papers with NO keywords: " + str(count_empty) + "/" + str(df.shape[0]) ) # most were empty
print( "Papers with keywords: " + str(count_non_empty) + "/" + str(df.shape[0]) )

Stop words retrieved from "stop_words.csv"!
Non-generated keywords retrieved from "non_generated_keywords.csv"!
Papers with NO keywords: 24312/26806
Papers with keywords: 0/26806


In [ ]:
df.to_csv("/content/drive/MyDrive/checkpoint_2_df.csv", encoding='utf-8', index=False)

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/checkpoint_11_df.csv")
len(df)

26806

In [ ]:
# Getting the article language and institution (only way, for the language at least) ...

# GIVES "HTTP Error 429: Too Many Requests" FOR 26806 articles ...

import urllib.request
from bs4 import BeautifulSoup

def get_page(url):
  fp = urllib.request.urlopen(url)
  mybytes = fp.read()
  html_page = mybytes.decode("utf8")
  fp.close()
  soup = BeautifulSoup(html_page)
  return soup

# def get_institution(row):
#   soup = get_page( row[ "url" ] )
#   # Here extracting paragraphs from the abstract ... yet maybe not all of them have paragraphs

#   # for i in soup.find("div", {"id": "abstract"}).find_all("strong", {"class": "sub-title"}):
#   #   p = i.parent
#   #   i.extract()
#   #   print( "\"" + str( p.text ).strip() + "\"" )
#   obj = soup.find("meta", {"name": "citation_author_institution"})
#   if ( obj != None ):
#     return obj['content']
#   return ""

# def get_language(row):
#   soup = get_page( row[ "url" ] )
#   return soup.find("meta", {"name": "citation_language"})['content']

# df['institution'] = df.apply(get_institution, axis=1)
# df['language'] = df.apply(get_language, axis=1)

In [ ]:
# Getting all the journal data that is available though the issn number ...
import json

i = 0
row_num = len(df)
def get_issn_data ( row ):
  soup = get_page( "https://portal.issn.org/resource/ISSN/" + str(row["issn"]) )
  global i
  i = i + 1
  print( "Getting ISSN info "+str(i)+" / "+str(row_num) )

  container = soup.find(attrs={"id" : "tab0"})

  if ( container == None ):
    return "{}"

  info_container = container.find(attrs={"class" : "item-result-content-text"})
  spans = info_container.find_all('span')
  journal_data={}
  for span in spans:
    attr = span.text
    parent = span.parent
    span.extract()

    attr = attr.replace(":", "").strip().lower()
    value = parent.text.strip().lower()
    if attr not in journal_data:
      journal_data[ attr ]=value
    else:
      if ( type( journal_data[ attr ] ).__name__ == 'list' ):
        journal_data[ attr ].append( value )
      else:
        journal_data[ attr ]=[ journal_data[ attr ], value ]
  return json.dumps(journal_data)

df["issn_data"]=df.apply(get_issn_data, axis=1)

Streaming output truncated to the last 5000 lines.
Getting ISSN info 21807 / 26806
Getting ISSN info 21808 / 26806
Getting ISSN info 21809 / 26806
Getting ISSN info 21810 / 26806
Getting ISSN info 21811 / 26806
Getting ISSN info 21812 / 26806
Getting ISSN info 21813 / 26806
Getting ISSN info 21814 / 26806
Getting ISSN info 21815 / 26806
Getting ISSN info 21816 / 26806
Getting ISSN info 21817 / 26806
Getting ISSN info 21818 / 26806
Getting ISSN info 21819 / 26806
Getting ISSN info 21820 / 26806
Getting ISSN info 21821 / 26806
Getting ISSN info 21822 / 26806
Getting ISSN info 21823 / 26806
Getting ISSN info 21824 / 26806
Getting ISSN info 21825 / 26806
Getting ISSN info 21826 / 26806
Getting ISSN info 21827 / 26806
Getting ISSN info 21828 / 26806
Getting ISSN info 21829 / 26806
Getting ISSN info 21830 / 26806
Getting ISSN info 21831 / 26806
Getting ISSN info 21832 / 26806
Getting ISSN info 21833 / 26806
Getting ISSN info 21834 / 26806
Getting ISSN info 21835 / 26806
Getting ISSN info 218

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/checkpoint_3_df.csv")

Mounted at /content/drive


In [ ]:
df["doi"]

,doi
0,10.1371/journal.pone.0039966
1,10.1097/INF.0000000000001271
2,10.1016/j.jinf.2012.02.012
3,10.5588/ijtld.10.0497
4,10.1371/journal.pmed.1001536
...,...
26801,10.1111/j.1742-6723.2005.00781.x
26802,10.1017/s1047951100007186
26803,10.1308/147870811X13137608454939
26804,10.1016/j.jclinane.2006.03.009


In [ ]:
df["references"].describe()
# NEXT WEEK WE MEET IN PERSON - SEE THE MICROSOFT CALENDAR OR SOMETHING ...

# use SCOPUS/OpenAlex to get references for the PMID papers that do NOT have em

# there is NO NEED for a citation network that contains ALL the PMIDs you have ...
# basically, each CSV file that Opeo gave you IS literature review, and you make a citation network for each review
# you ONLY put the title/citation of each reference as a node (cause they have to do NLP tasks on it, cannot just use PMID) (maybe full text/abstract) - better to just use the title/citation, so you dont have to retrieve absrtact/full text for all the references
# only include references (citations) that appear more than once in the current literature review (CSV file - no global citation network, just per literature review) = THE METHOD THAT OPEO SHOWED YOU WITH THE TABLE "number of papers that appear in this article that apperar in at least one more article" (all is done in a pool of articles that belong to the same literature review CSV file)
# sentance-embeddings for the citations/titles used
# only now use graph NN - search for pretrained ones in the paper they sent you about citation networks ... if its huggingface you canjust download it and use it ...

# citation snowballing - more advanced than "number of papers that appear in this article that apperar in at least one more article" - goes recursivly though the references of this article, then through the references of the references article ... or it connects multiple article nodes to the citations that were extracted already ...
# pretrained network embeddings survey on large scale citation networks - takes a citation network into an embedding space
# allan ai embedding model = https://allenai.org/blog/specter2-adapting-scientific-document-embeddings-to-multiple-fields-and-task-formats-c95686c06567

# Some papers about using GNN and network embedding over citation networks
# https://ieeexplore.ieee.org/abstract/document/10759845
# https://link.springer.com/article/10.1007/s11192-021-04196-3
# https://link.springer.com/article/10.1007/s11192-024-05067-3
# https://link.springer.com/article/10.1007/s11192-024-05109-w


,references
count,26806
unique,6252
top,[]
freq,20554


In [ ]:
# This should function too, yet I don't know if it's ok to bypass HTTP 403: "Forbidden" by changing the 'User-Agent' ... kinda the only way to get the full text/html of the article ...

# def get_pmc_data (row):
#   if ( row["pmc"] == None ):
#     return None

#   from urllib.request import Request, urlopen
#   site = "https://pmc.ncbi.nlm.nih.gov/articles/PMC"+row["pmc"]+"/"
#   hdr = {'User-Agent': 'Mozilla/5.0'} # Eighter this, or it detects that it is not a regular browser and restricts access ... Is this a problem?
#   req = Request(site,headers=hdr)
#   page = urlopen(req)
#   soup = BeautifulSoup(page, 'html.parser')

#   # container = soup.find(attrs={"class" : "body main-article-body"}) # the article without the title and so on ...
#   article = soup.find("article")
#   return article

# def get_pmc_data_html (row):
#   article = get_pmc_data(row)
#   if ( article != None ):
#     return str(article)
#   else:
#     return None

# def get_pmc_data_text (row):
#   article = get_pmc_data(row)
#   if ( article != None ):
#     return article.get_text()
#   else:
#     return None

# df["full_article_html"]=df.apply(get_pmc_data_html, axis=1)
# df["full_article_text"]=df.apply(get_pmc_data_text, axis=1)

In [ ]:
# Remove duplicated (not sure if necessary)

df = df.astype(str)
df.drop_duplicates(keep=False, inplace=True)

In [ ]:
# Saving only PMID/citation data to a CSV file. This file is meant for creating a citation network ...
# df[["pmid", "references"]].to_csv("/content/drive/MyDrive/citations.csv", encoding='utf-8', index=False)

# Saving the whole dataframe data to a CSV too ...
df.to_csv("/content/drive/MyDrive/extracted_features.csv", encoding='utf-8', index=False)

In [ ]:
# Printing the full article data (disable if you have tons of articles) ...

# def print_row ( iter ):
#   print("{:<20} {:<20}".format("Attribute", "Value")); print("-" * 41)
#   for i in range(0, len( df.columns ) ):
#     print("{:<20} {:<20}".format(str(df.columns[i]), str(df.iloc[iter].iloc[i])) )
#   print("\n\n")

# for k in range(0, df.shape[0] ):
#   print_row(k)

In [ ]:
# So, those are the features that were kept so far ...

print(df.columns.values)

In [ ]:
# This adds the columns of "df" to "csv_df" without adding any data ... not necessary, it is done when data is added ...

# for column in df.columns.values:
#   if ( column in [ "pmid", "abstract", "title" ] ):
#     continue
#   csv_df[ column ] = None

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/checkpoint_12_df.csv")
df = df.astype(str)

In [ ]:
df.columns

Index(['abstract', 'authors', 'chemicals', 'citation', 'doi', 'history',
       'issn', 'issue', 'journal', 'keywords', 'mesh', 'pmc', 'pmid',
       'publication_types', 'title', 'url', 'volume', 'volume_issue', 'xml',
       'year', 'author_first', 'references', 'issn_data'],
      dtype='object')

In [ ]:
df["history"]

,history
0,"{'received': datetime.datetime(2012, 3, 8, 0, ..."
1,"{'pubmed': datetime.datetime(2016, 10, 21, 0, ..."
2,"{'received': datetime.datetime(2011, 12, 21, 0..."
3,"{'entrez': datetime.datetime(2011, 3, 15, 0, 0..."
4,"{'received': datetime.datetime(2013, 4, 3, 0, ..."
...,...
26801,"{'pubmed': datetime.datetime(2005, 11, 24, 0, ..."
26802,"{'pubmed': datetime.datetime(1998, 12, 17, 0, ..."
26803,"{'entrez': datetime.datetime(2011, 10, 19, 0, ..."
26804,"{'received': datetime.datetime(2005, 10, 12, 0..."


In [ ]:
df["references"].describe()

,references
count,26806
unique,20942
top,[]
freq,5856


In [ ]:
!pip install demjson3
from demjson3 import decode

avg_citations = 0
total_non_null_citations = 0
def compute_average_citation_count(row):
  global avg_citations
  global total_non_null_citations
  if ( str(row["references"]) == "[]" ):
    return

  total_non_null_citations = total_non_null_citations + 1
  lngth = len(decode(row["references"]))
  print(lngth)
  avg_citations = avg_citations + lngth

df.apply(compute_average_citation_count, axis=1)

print( f'The total {total_non_null_citations} non-null citations have {round( avg_citations / total_non_null_citations )} citations on average.' )

15
10
29
32
16
11
10
64
26
32
71
1
40
10
2
6
12
41
3
25
39
20
20
20
11
25
1
13
7
24
29
8
16
15
9
32
25
25
5
31
28
15
12
12
12
30
5
7
52
18
7
51
12
10
56
15
20
14
26
24
19
6
2
83
13
29
8
14
26
30
24
21
75
7
22
27
23
26
6
12
10
21
15
15
26
40
14
27
14
30
25
15
15
25
11
14
4
6
22
12
22
40
9
28
12
22
26
25
19
18
8
14
7
18
6
88
5
23
3
5
23
9
9
36
6
13
10
43
11
13
51
7
17
1
20
17
5
7
2
25
22
10
13
12
25
23
9
8
10
25
8
31
101
58
16
13
7
11
21
17
6
7
33
17
13
23
5
18
59
13
8
23
4
5
1
4
16
4
11
24
20
8
28
7
8
27
7
15
25
8
15
24
12
19
1
7
4
17
10
11
23
10
5
9
12
10
11
24
3
94
28
4
21
6
11
10
16
10
8
22
7
22
18
9
11
2
24
22
25
20
8
25
27
3
29
24
7
13
25
7
24
7
274
4
13
14
13
14
17
17
18
21
5
9
49
9
7
21
14
4
1
27
9
10
19
10
11
25
13
7
21
3
7
37
37
9
15
9
25
28
16
2
4
33
10
23
3
8
14
24
8
10
19
37
34
3
15
8
5
9
25
40
4
28
15
19
26
10
26
44
38
7
31
17
51
3
116
9
1
6
25
7
3
1
24
11
10
12
15
3
14
11
25
31
4
14
8
29
19
11
6
19
121
8
10
22
35
23
2
18
4
15
43
24
4
112
36
123
16
6
36
5
4
11
32
11
27
14
7

KeyboardInterrupt: 

In [ ]:
csv_df = csv_df.astype(str)

In [ ]:
from IPython.display import clear_output
how_many_were_lost = 0
def get_data_by_pmid(row):
  try:
    pmid=row["PMID"]
    clear_output(wait=True)
    print( df[ df['pmid'] == str(pmid) ] )
    print( str(pmid) )
    row_iter = df.index[df['pmid'] == str(pmid)].tolist()[0]
    for column in df.columns.values:
      if ( column in [ "pmid", "abstract", "title" ] ):
        continue
      row[ column ] = df.loc[ row_iter ][ column ]
  except:
    global how_many_were_lost
    how_many_were_lost = how_many_were_lost + 1
  return row

csv_df = csv_df.apply(get_data_by_pmid, axis=1)

Streaming output truncated to the last 5000 lines.
                                                citation  \
26673  Chen L and Zhang Z. Bedside ultrasonography fo...   

                                       doi  \
26673  10.3978/j.issn.2223-4292.2015.05.04   

                                                 history       issn issue  \
26673  {'entrez': datetime.datetime(2015, 10, 6, 0, 0...  2223-4292     4   

                      journal  \
26673  Quant Imaging Med Surg   

                                                keywords  ...  \
26673  ['Critical care', 'lung point', 'pneumothorax ...  ...   

                                       publication_types  \
26673  {'D016428': 'Journal Article', 'D016454': 'Rev...   

                                                   title  \
26673  Bedside ultrasonography for diagnosis of pneum...   

                                            url volume volume_issue  \
26673  https://ncbi.nlm.nih.gov/pubmed/26435925      5         5(4)  

In [ ]:
df[ df["pmid"] == "12131381" ]

,abstract,authors,chemicals,citation,doi,history,issn,issue,journal,keywords,...,publication_types,title,url,volume,volume_issue,xml,year,author_first,references,issn_data


In [ ]:
len(df)

26806

In [ ]:
csv_df["PMID"].describe()

,PMID
count,105521
unique,26807
top,12831422
freq,8


In [ ]:
csv_df[ csv_df["PMID"] == 12131381 ]

,PMID,Status,Review Title,Review Criteria,Title,Abstract,text,rank,ada_embedding,prediction,...,hard4,hard5,score,f_score,c_score,soft1,soft2,soft3,soft4,soft5
18922,12131381,0,Triage tools for detecting cervical spine inju...,We included all retrospective and prospective ...,Spinal injuries in motorcycle crashes: pattern...,BACKGROUND: The purpose of this study was to d...,BACKGROUND: The purpose of this study was to d...,1743.0,"[-0.0005082697025500238, -0.003034350229427218...",0.828431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26954,12131381,0,Triage tools for detecting cervical spine inju...,We included all retrospective and prospective ...,Spinal injuries in motorcycle crashes: pattern...,BACKGROUND: The purpose of this study was to d...,BACKGROUND: The purpose of this study was to d...,1743.0,"[-0.0005082697025500238, -0.003034350229427218...",0.828431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34986,12131381,0,Triage tools for detecting cervical spine inju...,We included all retrospective and prospective ...,Spinal injuries in motorcycle crashes: pattern...,BACKGROUND: The purpose of this study was to d...,BACKGROUND: The purpose of this study was to d...,1260.0,"[-0.0005082697025500238, -0.003034350229427218...",0.828431,...,0.5,0.5,0.500000,0.664216,0.734865,NaN,NaN,NaN,NaN,NaN
43018,12131381,0,Triage tools for detecting cervical spine inju...,We included all retrospective and prospective ...,Spinal injuries in motorcycle crashes: pattern...,BACKGROUND: The purpose of this study was to d...,BACKGROUND: The purpose of this study was to d...,1315.0,"[-0.0005082697025500238, -0.003034350229427218...",0.828431,...,NaN,NaN,0.463815,0.646123,0.725819,0.463815,0.463815,0.463815,0.463815,0.463815


In [ ]:
# Save the newly aquired information to flies that match the source data filenames, withing the "Feature_extraction_results" folder

directory="/content/drive/MyDrive/Feature_extraction_results"
if not os.path.exists(directory):
    os.makedirs(directory)

filenames = csv_df['Filename'].unique()
for filename in filenames:
  df_filename = csv_df.loc[csv_df['Filename'] == filename]
  df_filename = df_filename.drop('Filename', axis=1)
  df_filename.to_csv("/content/drive/MyDrive/Feature_extraction_results/" + filename, encoding='utf-8', index=False)

In [ ]:
print(how_many_were_lost) # so, 4 articles were lost somewhere ... no clue why ... good thing it's just 4 i guess ...

4


In [ ]:
# I did not code this and it seemed interesting ... why is there a try/catch? are not all reviews public? might some have been deleted?

# num = 0
# while num < CD012768.shape[0]:
#   pmid = pmids[num]
#   try:
#     article = fetch.article_by_pmid(pmid)
#     titles[pmid] = article.title
#     abstracts[pmid] = article.abstract
#   except:
#     pass
#   else:
#     print(num)
#     num += 1